In [16]:
from langchain.document_loaders import DirectoryLoader

directory = 'input_dataset'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

13

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


42


In [19]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [20]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [22]:
query = "What was the cost of membership in 2020?"
matching_docs = db.similarity_search(query)

matching_docs[0]

Document(page_content='USD\n\nFirst person:\t1,200\n\nAdditional family member:\t200\n\nSixth person & above:\t0\n\nChildren under five:\t0\n\nb\n\n\n\nNON\n\n\n\nRESIDENT MEMBERSHIP\n\nUSD\n\nFirst person:\t800\n\nAdditional family member:\t150\n\nSixth person & above:\t0\n\nChildren under five:\t0\n\nc- FAMILY-PLUS SUBSCRIPTION A new subscription plan, linked to your Membership and dedicated to non-family Members. The plan is applicable to Resident and Non-Resident members. Please find below relevant fees and conditions.\n\nUSD Spouses’ parent:               \t$450 Living-in relative:               \t$450 Children’s caregiver:       \t$450 Single friend:                     \t$800 Accompanying caregiver*:\t$0\n\nTo help disabled persons or supervise children when no parent\n\nor guardian can be present, without using Club facilities.\n\nd\n\n\n\nDEACTIVATED MEMBEFRSHIP\n\n$500, for both Resident and Non-Resident Memberships.\n\n3. PAYMENTS MODES', metadata={'source': 'input_dataset\\

In [23]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()

In [24]:
import os
from credentials_hander import OPENAI_API_KEY
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [33]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "What was the cost of memberships in 2022?"
# matching_docs = db.similarity_search(query)
# answer =  chain.run(input_documents=matching_docs, question=query)
# answer

In [34]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'The cost of memberships in 2022 is not provided in the given context.'